In [1]:
import numpy as np 
import pandas as pd 
from data_processing import generate_vocab, process_data, create_dataloaders 
from model import get_pretrained_emb, EncoderDecoder, EncoderRNN, DecoderRNN, DecoderSimpleRNN, EncoderSimpleRNN, \
    Attention, DecoderAttnRNN, DecoderRNNV2, EncoderDecoderAttention
from train_eval import count_parameters, summarize_results, \
    plot_single_learning_curve, load_experiment_log
from train_eval import train_and_eval, train_and_eval_attn 
import importlib
import pickle as pkl 
import torch

In [2]:
# model identification
MODEL_NAME = 'zh-seq2seq-rnn-attention'
SRC_LANG = 'zh'
TARG_LANG = 'en'

# data processing params  
SRC_MAX_SENTENCE_LEN = 10
TARG_MAX_SENTENCE_LEN = 10
SRC_VOCAB_SIZE = 30000 #30000
TARG_VOCAB_SIZE = 30000 #30000

# model architecture params 
NUM_LAYERS = 2 #2 
ENC_HIDDEN_DIM = 300 
DEC_HIDDEN_DIM = ENC_HIDDEN_DIM #2 * ENC_HIDDEN_DIM 
TEACHER_FORCING_RATIO = 1
CLIP_GRAD_MAX_NORM = 1
ENC_DROPOUT = 0 # to actually implement
DEC_DROPOUT = 0 # to actually implement

# training params  
BATCH_SIZE = 32 #32
NUM_EPOCHS = 500
LR = 0.0005 # 0.0005
OPTIMIZER = 'Adam'
LAZY_TRAIN = True

In [3]:
# store as dict to save to results later 
params = {'model_name': MODEL_NAME, 'src_lang': SRC_LANG, 'targ_lang': TARG_LANG, 
          'src_max_sentence_len': SRC_MAX_SENTENCE_LEN, 'targ_max_sentence_len': TARG_MAX_SENTENCE_LEN, 
          'src_vocab_size': SRC_VOCAB_SIZE, 'targ_vocab_size': TARG_VOCAB_SIZE, 
          'num_layers': NUM_LAYERS, 'enc_hidden_dim': ENC_HIDDEN_DIM, 'dec_hidden_dim': DEC_HIDDEN_DIM,
          'teacher_forcing_ratio': TEACHER_FORCING_RATIO, 'clip_grad_max_norm': CLIP_GRAD_MAX_NORM,
          'enc_dropout': ENC_DROPOUT, 'dec_dropout': DEC_DROPOUT, 
          'batch_size': BATCH_SIZE, 'num_epochs': NUM_EPOCHS, 'learning_rate': LR, 'optimizer': OPTIMIZER, 
          'lazy_train': LAZY_TRAIN} 

In [4]:
#vocab_test = generate_vocab(SRC_LANG, TARG_LANG, SRC_VOCAB_SIZE, TARG_VOCAB_SIZE)

In [5]:
#vocab['zh']['id2token'][987]

In [6]:
#vocab['zh']['token2id']['森林']

In [7]:
#vocab['en']['token2id']['activity']

In [8]:
#vocab['en']['id2token'][987]

In [9]:
# # takes a long time to process, save to pickle for reimport in future 
# vocab = generate_vocab(SRC_LANG, TARG_LANG, SRC_VOCAB_SIZE, TARG_VOCAB_SIZE)
# vocab_filename = "{}-{}-vocab.p".format(SRC_LANG, TARG_LANG)
# pkl.dump(vocab, open(vocab_filename, "wb"))

In [10]:
# reload from pickle 
vocab_filename = "{}-{}-vocab.p".format(SRC_LANG, TARG_LANG)
vocab = pkl.load(open(vocab_filename, "rb"))
data = process_data(SRC_LANG, TARG_LANG, vocab)
data_minibatch = process_data(SRC_LANG, TARG_LANG, vocab, sample_limit=BATCH_SIZE) 
data_minitrain = process_data(SRC_LANG, TARG_LANG, vocab, sample_limit=1000)

In [11]:
# # takes a long time to process, save to pickle for reimport in future 
# vocab = generate_vocab(SRC_LANG, TARG_LANG, SRC_VOCAB_SIZE, TARG_VOCAB_SIZE)
# vocab_filename = "{}-{}-vocab-fake.p".format(SRC_LANG, TARG_LANG)
# pkl.dump(vocab, open(vocab_filename, "wb"))

In [12]:
# vocab_filename = "{}-{}-vocab-fake.p".format(SRC_LANG, TARG_LANG)
# vocab = pkl.load(open(vocab_filename, "rb"))
# data = process_data(SRC_LANG, TARG_LANG, vocab)
# limited_data = process_data(SRC_LANG, TARG_LANG, vocab, sample_limit=BATCH_SIZE) 

In [13]:
# create dataloaders 
loaders_full = create_dataloaders(data, SRC_MAX_SENTENCE_LEN, TARG_MAX_SENTENCE_LEN, BATCH_SIZE)
loaders_minibatch = create_dataloaders(data_minibatch, SRC_MAX_SENTENCE_LEN, TARG_MAX_SENTENCE_LEN, BATCH_SIZE)
loaders_minitrain = create_dataloaders(data_minitrain, SRC_MAX_SENTENCE_LEN, TARG_MAX_SENTENCE_LEN, BATCH_SIZE)

In [14]:
# define model 

# encoder = EncoderRNN(enc_hidden_dim=ENC_HIDDEN_DIM, num_layers=NUM_LAYERS, src_max_sentence_len=SRC_MAX_SENTENCE_LEN,
#                      pretrained_word2vec=get_pretrained_emb(vocab[SRC_LANG]['word2vec'], vocab[SRC_LANG]['token2id']))
encoder = EncoderSimpleRNN(enc_hidden_dim=ENC_HIDDEN_DIM, num_layers=NUM_LAYERS, src_max_sentence_len=SRC_MAX_SENTENCE_LEN,
                           pretrained_word2vec=get_pretrained_emb(vocab[SRC_LANG]['word2vec'], vocab[SRC_LANG]['token2id']))

# decoder = DecoderRNN(dec_hidden_dim=DEC_HIDDEN_DIM, enc_hidden_dim=ENC_HIDDEN_DIM, num_layers=NUM_LAYERS,
#                       targ_vocab_size=TARG_VOCAB_SIZE, targ_max_sentence_len=TARG_MAX_SENTENCE_LEN, 
#                       pretrained_word2vec=get_pretrained_emb(vocab[TARG_LANG]['word2vec'], vocab[TARG_LANG]['token2id']))

# decoder = DecoderRNNV2(dec_hidden_dim=DEC_HIDDEN_DIM, enc_hidden_dim=ENC_HIDDEN_DIM, num_layers=NUM_LAYERS, 
#                        targ_vocab_size=TARG_VOCAB_SIZE, targ_max_sentence_len=TARG_MAX_SENTENCE_LEN, 
#                        pretrained_word2vec=get_pretrained_emb(vocab[TARG_LANG]['word2vec'], vocab[TARG_LANG]['token2id']))
# decoder = DecoderSimpleRNN(dec_hidden_dim=DEC_HIDDEN_DIM, enc_hidden_dim=ENC_HIDDEN_DIM, num_layers=NUM_LAYERS, 
#                            targ_vocab_size=TARG_VOCAB_SIZE, targ_max_sentence_len=TARG_MAX_SENTENCE_LEN, 
#                            pretrained_word2vec=get_pretrained_emb(vocab[TARG_LANG]['word2vec'], vocab[TARG_LANG]['token2id']))
decoder = DecoderAttnRNN(dec_hidden_dim=DEC_HIDDEN_DIM, enc_hidden_dim=ENC_HIDDEN_DIM, num_layers=NUM_LAYERS, 
                         targ_vocab_size=TARG_VOCAB_SIZE, src_max_sentence_len=SRC_MAX_SENTENCE_LEN, 
                         targ_max_sentence_len=TARG_MAX_SENTENCE_LEN, 
                         pretrained_word2vec=get_pretrained_emb(vocab[TARG_LANG]['word2vec'], vocab[TARG_LANG]['token2id']))

model = EncoderDecoderAttention(encoder, decoder, vocab[TARG_LANG]['token2id']) 

In [15]:
model, results = train_and_eval_attn(
    model=model, loaders_full=loaders_full, loaders_minibatch=loaders_minibatch, loaders_minitrain=loaders_minitrain, 
    params=params, vocab=vocab, print_intermediate=True, save_checkpoint=True, save_to_log=True, 
    lazy_eval=False, inspect_samples=1)

Epoch: 0.00, Train Loss: 10.13, Val Loss: 10.25, Train BLEU: 0.28, Val BLEU: 0.20
Sampling from training predictions...
Source: 我 真 喜欢 这些 东西 <EOS> <PAD> <PAD> <PAD> <PAD>
Reference: i love that kind of stuff . <EOS> <PAD>
Model: <SOS> the the the the the the the dormant dormant
Attention Weights: tensor([[0.1627, 0.1642, 0.1642, 0.1636, 0.1666, 0.1635, 0.0038, 0.0038, 0.0038,
         0.0038],
        [0.1628, 0.1643, 0.1641, 0.1636, 0.1666, 0.1636, 0.0037, 0.0037, 0.0037,
         0.0037],
        [0.1629, 0.1642, 0.1641, 0.1636, 0.1667, 0.1639, 0.0037, 0.0037, 0.0037,
         0.0037],
        [0.1628, 0.1641, 0.1640, 0.1636, 0.1667, 0.1638, 0.0037, 0.0037, 0.0037,
         0.0037],
        [0.1628, 0.1641, 0.1640, 0.1635, 0.1667, 0.1638, 0.0038, 0.0038, 0.0038,
         0.0038],
        [0.1628, 0.1640, 0.1640, 0.1635, 0.1667, 0.1638, 0.0038, 0.0038, 0.0038,
         0.0038],
        [0.1628, 0.1640, 0.1640, 0.1635, 0.1667, 0.1638, 0.0038, 0.0038, 0.0038,
         0.0038],
        [

Epoch: 4.00, Train Loss: 8.97, Val Loss: 9.77, Train BLEU: 0.31, Val BLEU: 0.21
Sampling from training predictions...
Source: 看到 这些 在 动 的 东西 了 吗 <EOS> <PAD>
Reference: but see all those different working things ? <EOS>
Model: <SOS> the the the the the the the the the
Attention Weights: tensor([[0.0957, 0.1038, 0.1073, 0.1077, 0.1068, 0.1051, 0.1016, 0.0925, 0.0798,
         0.0997],
        [0.0907, 0.1007, 0.1058, 0.1071, 0.1064, 0.1043, 0.1004, 0.0902, 0.0760,
         0.1183],
        [0.0884, 0.0986, 0.1041, 0.1056, 0.1050, 0.1031, 0.0992, 0.0892, 0.0749,
         0.1318],
        [0.0869, 0.0969, 0.1024, 0.1040, 0.1034, 0.1015, 0.0978, 0.0881, 0.0741,
         0.1448],
        [0.0860, 0.0959, 0.1013, 0.1028, 0.1023, 0.1005, 0.0969, 0.0875, 0.0737,
         0.1531],
        [0.0857, 0.0954, 0.1008, 0.1023, 0.1017, 0.0999, 0.0964, 0.0872, 0.0735,
         0.1571],
        [0.0856, 0.0952, 0.1005, 0.1020, 0.1014, 0.0997, 0.0962, 0.0871, 0.0736,
         0.1587],
        [0.0856, 0.0

Epoch: 8.00, Train Loss: 7.37, Val Loss: 9.17, Train BLEU: 0.30, Val BLEU: 0.20
Sampling from training predictions...
Source: 大部 大部分 部分 的 动物 也 都 生活 在 海洋
Reference: most of the animals are in the oceans .
Model: <SOS> the the the the the the the the the
Attention Weights: tensor([[0.0967, 0.0997, 0.0999, 0.0998, 0.1004, 0.1014, 0.1023, 0.1030, 0.1022,
         0.0947],
        [0.0848, 0.0957, 0.1015, 0.1047, 0.1060, 0.1076, 0.1077, 0.1059, 0.1010,
         0.0851],
        [0.0814, 0.0944, 0.1019, 0.1060, 0.1074, 0.1093, 0.1091, 0.1065, 0.1008,
         0.0833],
        [0.0806, 0.0941, 0.1020, 0.1064, 0.1078, 0.1097, 0.1094, 0.1065, 0.1006,
         0.0828],
        [0.0804, 0.0941, 0.1021, 0.1065, 0.1080, 0.1098, 0.1095, 0.1065, 0.1005,
         0.0826],
        [0.0804, 0.0941, 0.1021, 0.1066, 0.1080, 0.1099, 0.1095, 0.1065, 0.1004,
         0.0825],
        [0.0804, 0.0941, 0.1021, 0.1066, 0.1081, 0.1099, 0.1095, 0.1065, 0.1004,
         0.0825],
        [0.0804, 0.0941, 0.1021, 0.

Epoch: 12.00, Train Loss: 5.76, Val Loss: 8.75, Train BLEU: 0.28, Val BLEU: 0.19
Sampling from training predictions...
Source: 还有 这些 摇晃 着 旋转 转着 的 触角 <EOS> <PAD>
Reference: it &apos;s got tentacles dangling , swirling around like
Model: <SOS> the the the the the the the the the
Attention Weights: tensor([[0.0940, 0.0989, 0.1014, 0.1031, 0.1046, 0.1063, 0.1051, 0.1006, 0.0831,
         0.1029],
        [0.0380, 0.0463, 0.0498, 0.0515, 0.0521, 0.0516, 0.0506, 0.0446, 0.0304,
         0.5851],
        [0.0219, 0.0273, 0.0297, 0.0309, 0.0312, 0.0306, 0.0301, 0.0262, 0.0177,
         0.7546],
        [0.0174, 0.0219, 0.0238, 0.0248, 0.0250, 0.0245, 0.0241, 0.0210, 0.0142,
         0.8035],
        [0.0158, 0.0198, 0.0216, 0.0225, 0.0227, 0.0222, 0.0218, 0.0190, 0.0129,
         0.8218],
        [0.0150, 0.0188, 0.0204, 0.0213, 0.0215, 0.0210, 0.0207, 0.0180, 0.0122,
         0.8311],
        [0.0146, 0.0183, 0.0199, 0.0207, 0.0209, 0.0205, 0.0202, 0.0176, 0.0119,
         0.8355],
        [0

Epoch: 16.00, Train Loss: 4.80, Val Loss: 8.92, Train BLEU: 0.33, Val BLEU: 0.21
Sampling from training predictions...
Source: 是 我 最 喜欢 的 因为 它 哪 都 能动
Reference: it &apos;s one of my favorites , because it
Model: <SOS> it the the the the the the the the
Attention Weights: tensor([[0.0918, 0.0977, 0.1001, 0.1014, 0.1025, 0.1032, 0.1039, 0.1046, 0.1035,
         0.0913],
        [0.0824, 0.0998, 0.1044, 0.1063, 0.1075, 0.1084, 0.1093, 0.1093, 0.1026,
         0.0698],
        [0.0801, 0.0997, 0.1052, 0.1073, 0.1086, 0.1094, 0.1103, 0.1101, 0.1020,
         0.0672],
        [0.0796, 0.0994, 0.1052, 0.1074, 0.1088, 0.1096, 0.1106, 0.1103, 0.1020,
         0.0672],
        [0.0794, 0.0992, 0.1051, 0.1075, 0.1089, 0.1097, 0.1106, 0.1104, 0.1019,
         0.0672],
        [0.0794, 0.0991, 0.1051, 0.1075, 0.1089, 0.1098, 0.1107, 0.1104, 0.1019,
         0.0672],
        [0.0793, 0.0991, 0.1052, 0.1075, 0.1090, 0.1098, 0.1107, 0.1104, 0.1019,
         0.0671],
        [0.0792, 0.0991, 0.1052, 0.

...

Epoch: 456.00, Train Loss: 0.02, Val Loss: 14.16, Train BLEU: 100.00, Val BLEU: 0.29
Sampling from training predictions...
Source: 和 我们 合作 的 人们 帮 我们 找到 了 新
Reference: people that have partnered with us have given us
Model: <SOS> people that have partnered with us have given us
Attention Weights: tensor([[0.0808, 0.0978, 0.1092, 0.1026, 0.1079, 0.0994, 0.1079, 0.0968, 0.0901,
         0.1075],
        [0.0799, 0.0975, 0.1096, 0.1018, 0.1073, 0.0981, 0.1072, 0.0989, 0.0903,
         0.1094],
        [0.0793, 0.0964, 0.1095, 0.1030, 0.1074, 0.0982, 0.1078, 0.1010, 0.0907,
         0.1066],
        [0.0797, 0.0981, 0.1075, 0.1029, 0.1059, 0.0995, 0.1061, 0.1023, 0.0948,
         0.1031],
        [0.0773, 0.0980, 0.1049, 0.1041, 0.1062, 0.1015, 0.1055, 0.1040, 0.0979,
         0.1006],
        [0.0726, 0.0966, 0.1034, 0.1045, 0.1073, 0.1037, 0.1061, 0.1057, 0.1006,
         0.0995],
        [0.0688, 0.0934, 0.1023, 0.1040, 0.1083, 0.1035, 0.1081, 0.1095, 0.1026,
         0.0994],
        [0

Epoch: 460.00, Train Loss: 0.02, Val Loss: 14.17, Train BLEU: 100.00, Val BLEU: 0.29
Sampling from training predictions...
Source: 其实 实地 地球 上 最长 的 山脉 都 在 海洋
Reference: and in the oceans , there are the longest
Model: <SOS> and in the oceans , there are the longest
Attention Weights: tensor([[0.0814, 0.1024, 0.1146, 0.0996, 0.1039, 0.1016, 0.1077, 0.0960, 0.0941,
         0.0987],
        [0.0835, 0.1042, 0.1139, 0.0957, 0.1027, 0.0993, 0.1086, 0.0929, 0.0958,
         0.1034],
        [0.0801, 0.1028, 0.1140, 0.0967, 0.1030, 0.0997, 0.1088, 0.0940, 0.0970,
         0.1038],
        [0.0747, 0.1001, 0.1123, 0.0993, 0.1041, 0.1015, 0.1085, 0.0986, 0.1001,
         0.1007],
        [0.0729, 0.0994, 0.1092, 0.1025, 0.1034, 0.1037, 0.1075, 0.1031, 0.0988,
         0.0996],
        [0.0667, 0.0977, 0.1119, 0.1033, 0.1052, 0.1042, 0.1098, 0.1048, 0.0992,
         0.0972],
        [0.0681, 0.0962, 0.1093, 0.1036, 0.1037, 0.1039, 0.1084, 0.1063, 0.1005,
         0.1000],
        [0.0657, 0.0952

Epoch: 464.00, Train Loss: 0.02, Val Loss: 14.19, Train BLEU: 100.00, Val BLEU: 0.29
Sampling from training predictions...
Source: 底下 这些 都 是 <UNK> 它们 上上 上上下下 上下 下下
Reference: it &apos;s got these fishing <UNK> on the bottom
Model: <SOS> it &apos;s got these fishing <UNK> on the bottom
Attention Weights: tensor([[0.0866, 0.0905, 0.0879, 0.0978, 0.1554, 0.0983, 0.1156, 0.0868, 0.0857,
         0.0955],
        [0.0858, 0.0894, 0.0874, 0.0996, 0.1530, 0.1003, 0.1128, 0.0854, 0.0861,
         0.1003],
        [0.0826, 0.0867, 0.0862, 0.1018, 0.1602, 0.0991, 0.1074, 0.0840, 0.0852,
         0.1067],
        [0.0811, 0.0904, 0.0883, 0.1030, 0.1462, 0.1037, 0.1116, 0.0878, 0.0891,
         0.0989],
        [0.0828, 0.0938, 0.0889, 0.1018, 0.1362, 0.1058, 0.1156, 0.0929, 0.0933,
         0.0889],
        [0.0792, 0.0926, 0.0885, 0.1003, 0.1415, 0.1014, 0.1187, 0.0944, 0.0942,
         0.0891],
        [0.0730, 0.0918, 0.0878, 0.0949, 0.1363, 0.1013, 0.1306, 0.1023, 0.1001,
         0.0818],
  

Epoch: 468.00, Train Loss: 0.02, Val Loss: 14.20, Train BLEU: 100.00, Val BLEU: 0.29
Sampling from training predictions...
Source: 大卫 <UNK> 这位 是 比尔 <UNK> 我 是 大卫 <UNK>
Reference: this is bill lange . i &apos;m dave gallo
Model: <SOS> this is bill lange . i &apos;m dave gallo
Attention Weights: tensor([[0.0895, 0.1238, 0.0898, 0.0814, 0.1320, 0.1320, 0.0779, 0.0745, 0.0784,
         0.1208],
        [0.0898, 0.1179, 0.0948, 0.0859, 0.1333, 0.1224, 0.0790, 0.0773, 0.0823,
         0.1173],
        [0.0963, 0.1081, 0.0961, 0.0944, 0.1330, 0.1179, 0.0802, 0.0819, 0.0891,
         0.1032],
        [0.0883, 0.1100, 0.0942, 0.0941, 0.1434, 0.1244, 0.0751, 0.0776, 0.0883,
         0.1046],
        [0.0872, 0.1061, 0.0998, 0.1018, 0.1444, 0.1228, 0.0775, 0.0813, 0.0914,
         0.0878],
        [0.0768, 0.1222, 0.0908, 0.0933, 0.1636, 0.1392, 0.0652, 0.0698, 0.0840,
         0.0952],
        [0.0793, 0.1214, 0.0903, 0.0912, 0.1692, 0.1364, 0.0649, 0.0695, 0.0839,
         0.0939],
        [0.07

KeyboardInterrupt: 

In [ ]:
summarize_results(load_experiment_log())[['dt_created', 'num_epochs', 'learning_rate', 'clip_grad_max_norm', 'val_loss']].head()

In [ ]:
plot_single_learning_curve(results)

In [ ]:
# Epoch: 199.00, Train Loss: 0.32, Val Loss: 13.19, Train BLEU: 98.94, Val BLEU: 0.27
plot_single_learning_curve(results)

In [ ]:
# with attention energies = v_broadcast.bmm(torch.tanh(self.attn(concat)).transpose(1, 2)) # switched order  
# Epoch: 199.00, Train Loss: 0.63, Val Loss: 12.82, Train BLEU: 92.05, Val BLEU: 0.38
plot_single_learning_curve(results)

In [ ]:
for i, token in enumerate(vocab[SRC_LANG]['id2token']): 
    if i < 20: 
        print("{}: {}".format(i, token))

In [ ]:
for i, token in enumerate(vocab[TARG_LANG]['id2token']): 
    if i < 20: 
        print("{}: {}".format(i, token))

In [ ]:
import torch
x = torch.arange(0, 3*5*10).view(3, 5, 10)
print(x)
y = x[1:, :, :]
print(y)
z = y.view(-1, 10)
print(z)

In [ ]:
t = torch.arange(0, 2*5).view(5, 2)
print(t)
u = t.contiguous().view(-1)
print(u)
v = t.permute(1, 0)
print(v)
w = v.contiguous().view(-1)
print(w)

In [ ]:
a = torch.arange(0, 2*1*300)
print(a)
b = a.view(-1, 1, 300)
print(b.size())

In [ ]:
for i, (src_idxs, targ_idxs, src_lens, targ_lens) in enumerate(full_loaders['train']):
#     print(i)
#     print(src_idxs.size())
#     print(src_idxs)
#     print(src_lens)
#     print(targ_idxs.size())
#     print(targ_idxs)
#     print(targ_lens)
    id2token = vocab[SRC_LANG]['id2token']
    test_tensor = src_idxs
    list_of_lists = test_tensor.numpy().astype(int).tolist()
    to_token = lambda l: ' '.join([id2token[idx] for idx in l])
    list_of_lists_tokens = [to_token(l) for l in list_of_lists] 
    break 